In [1]:
import torch
import onnx
import onnxruntime as ort
import netron

In [2]:
print(torch.__version__)
print(onnx.__version__)
print(ort.get_device())
print(ort.__version__)

2.7.0+cu126
1.18.0
GPU
1.22.0


In [3]:
MODEL_PATH = "./result/best.pt" # 모델 경로
ONNX_PATH = "./onnx/panns_cnn14.onnx" # 오닉스 저장경로
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu' # cuda 사용 가능여부 확인

In [4]:
print(DEVICE)

cuda


In [5]:
# 모델 로드
model = torch.load(MODEL_PATH, weights_only=False)
model.to(DEVICE)

Cnn14(
  (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_block1): ConvBlock(
    (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block2): ConvBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block3): ConvBlock(
    (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1),

In [6]:
# 더미 생성: (batch, 1, time, mel_bins)
dummy_input = torch.randn(1, 1, 101, 64, device=DEVICE)

In [7]:
print(dummy_input.shape)

torch.Size([1, 1, 101, 64])


In [8]:
# onnx 모델 변환
print("ONNX 변환중...")

torch.onnx.export(model, # PyTorch 모델
                dummy_input, # 입력 텐서 (튜플도 가능)
                ONNX_PATH, # 저장할 파일 경로
                export_params=True, # 모델 파라미터 포함
                opset_version=11, # onnx의 연산자 버전
                input_names=['input'], # 입력 이름
                output_names=['output'], # 출력 이름
                dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}} # 배치 크기를 동적으로 지정
)

print(f"ONNX 변환완료'{ONNX_PATH}'에 저장됨")

ONNX 변환중...
ONNX 변환완료'./onnx/panns_cnn14.onnx'에 저장됨


In [9]:
model = onnx.load(ONNX_PATH)
print(onnx.helper.printable_graph(model.graph))

graph main_graph (
  %input[FLOAT, batch_sizex1x101x64]
) initializers (
  %bn0.weight[FLOAT, 64]
  %bn0.bias[FLOAT, 64]
  %bn0.running_mean[FLOAT, 64]
  %bn0.running_var[FLOAT, 64]
  %fc1.weight[FLOAT, 2048x2048]
  %fc1.bias[FLOAT, 2048]
  %fc_audioset.weight[FLOAT, 5x2048]
  %fc_audioset.bias[FLOAT, 5]
  %onnx::Conv_136[FLOAT, 64x1x3x3]
  %onnx::Conv_137[FLOAT, 64]
  %onnx::Conv_139[FLOAT, 64x64x3x3]
  %onnx::Conv_140[FLOAT, 64]
  %onnx::Conv_142[FLOAT, 128x64x3x3]
  %onnx::Conv_143[FLOAT, 128]
  %onnx::Conv_145[FLOAT, 128x128x3x3]
  %onnx::Conv_146[FLOAT, 128]
  %onnx::Conv_148[FLOAT, 256x128x3x3]
  %onnx::Conv_149[FLOAT, 256]
  %onnx::Conv_151[FLOAT, 256x256x3x3]
  %onnx::Conv_152[FLOAT, 256]
  %onnx::Conv_154[FLOAT, 512x256x3x3]
  %onnx::Conv_155[FLOAT, 512]
  %onnx::Conv_157[FLOAT, 512x512x3x3]
  %onnx::Conv_158[FLOAT, 512]
  %onnx::Conv_160[FLOAT, 1024x512x3x3]
  %onnx::Conv_161[FLOAT, 1024]
  %onnx::Conv_163[FLOAT, 1024x1024x3x3]
  %onnx::Conv_164[FLOAT, 1024]
  %onnx::Conv_166

In [10]:
try:
    onnx.checker.check_model(model)
    print("모델 검증 성공")
except onnx.checker.ValidationError as e:
    print("모델 검증 실패")
    print(e)

모델 검증 성공


In [11]:
# 모델 시각화
netron.start (ONNX_PATH)

Serving './onnx/panns_cnn14.onnx' at http://localhost:8080


('localhost', 8080)